In [ ]:
import re
import ast
import numpy as np

PATH = '/Users/xuyaozhang/Desktop/AdventofCode2022/input/'

def read_txt(filename):
    res = []
    instructions = ""
    
    with open(filename) as data_file:
            
        lines = data_file.read().split('\n') 
        instructions = lines.pop()
        lines = [[*lst] for lst in lines]
        lengths = [len(lst) for lst in lines]
        
        lines.pop()
            
    return lines, instructions, max(lengths)

        
def fill_arr(arr, num):
    arr += ['0']*num
    return arr

def cal_ans1(row, column, f):
    row += 1
    column += 1
    #Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^)
    score = {'R': 0, 'D': 1, 'L': 2, 'U': 3}
    return 1000*row + 4*column + score[f]

def cal_ans2(row, column, ori_r, ori_c):
    row += 1
    column += 1
    #Facing is 0 for right (>), 1 for down (v), 2 for left (<), and 3 for up (^)
    score = {(0, 1): 0, (1, 0): 1, (0,-1): 2, (-1, 0): 3}
    return 1000*row + 4*column + score[(ori_r, ori_c)]
    

def move(row, column, arr, tarr, action, step, length_c, length_r):
    
    for s in range(step):
        if action == 'R':
            column += 1
            if column == length_c or arr[row][column] == '0':
                tile_index = arr[row].index('.')
                wall_index = arr[row].index('#')
                if tile_index < wall_index:
                    column = tile_index
                else:
                    return row, column-1
                
            if arr[row][column] == '#':
                return row, column-1
            
        if action == 'L':
            column -= 1
            if column == -1 or arr[row][column] == ' ':
                tile_index = length_c - 1 - arr[row][::-1].index('.')
                wall_index = length_c - 1 - arr[row][::-1].index('#')
                if tile_index > wall_index:
                    column = tile_index
                else:
                    return row, column+1
            if arr[row][column] == '#':
                return row, column+1
            
        if action == 'U':
            row -= 1
            if row == -1 or tarr[column][row] == '0' or tarr[column][row] == ' ':
                tile_index = length_r - 1 - tarr[column][::-1].index('.')
                if '#' in tarr[column]:
                    
                    wall_index = length_r - 1 - tarr[column][::-1].index('#')
                    if tile_index > wall_index:
                        row = tile_index
                    else:
                        return row+1, column
                else:
                    row = tile_index
            if arr[row][column] == '#':
                return row+1, column
            
        if action == 'D':
            row += 1
            if row == length_r or tarr[column][row] == ' ' or tarr[column][row] == '0':
                tile_index = tarr[column].index('.')
                if '#' in tarr[column]:
                    wall_index = tarr[column].index('#')
                    if tile_index < wall_index:
                        row = tile_index
                    else:
                        return row-1, column
                else:
                    row = tile_index
                
            if arr[row][column] == '#':
                return row-1, column
            
        
            
            
            
    return row, column


# better than move() as we don't need consider four directions separately and find index from the list many times
def move2(row, column, arr, action, step, ori_r, ori_c):
    """
    Structure of the cube: 
      12
      3
     45
     6
    """
    if action == "R": 
        ori_r, ori_c = ori_c, -ori_r
    else: 
        ori_r, ori_c = -ori_c, ori_r
    
    for s in range(step):
        initial_r = ori_r
        initial_c = ori_c
        initial_row = row
        initial_column = column
        row += ori_r
        column += ori_c
        
        if row < 0 and 50 <= column < 100 and ori_r == -1: #In face 1, move up to face 6 transpose, direction changes to R
            ori_r = 0
            ori_c = 1
            row = column + 100
            column = 0
            
        elif column == 49 and 0 <= row < 50 and ori_c == -1: #In face 1, move left to face 4, direction changes to R
            ori_c = 1
            row = 149 - row
            column = 0
            
        elif row < 0 and 100 <= column < 150 and ori_r == -1: #In face 2, move up to face 6
            row = 199
            column -= 100
            
        elif column >= 150 and 0 <= row < 50 and ori_c == 1: #In face 2, move right to face 5, direction changes to L
            ori_c = -1
            row = 149 - row
            column = 99
            
        elif row == 50 and 100 <= column < 150 and ori_r == 1: #In face 2, move down to face 3 transpose, direction changes to L
            ori_r = 0
            ori_c = -1
            row = column - 50
            column = 99
            
        elif column == 100 and 50 <= row < 100 and ori_c == 1: #In face 3, move right to face 2 transpose, direction changes to U
            ori_r = -1
            ori_c = 0 
            column = row + 50
            row = 49
            
        elif column == 49 and 50 <= row < 100 and ori_c == -1: #In face 3, move left to face 4 transpose, direction changes to D
            ori_r = 1
            ori_c = 0
            column = row - 50
            row = 100
        
        elif row == 99 and 0 <= column < 50 and ori_r == -1: #In face 4, move up to face 3 transpose, direction changes to R
            ori_r = 0
            ori_c = 1
            row = column + 50
            column = 50
            
        elif column < 0 and 100 <= row < 150 and ori_c == -1: #In face 4, move left to face 1, direction changes to R
            ori_c = 1
            row = 149 - row
            column = 50
            
        elif column == 100 and 100 <= row < 150 and ori_c == 1: #In face 5, move right to face 2, direction changes to L
            ori_c = -1
            row = 149 - row
            column = 149
            
        elif row == 150 and 50 <= column < 100 and ori_r == 1: #In face 5, move down to face 6 transpose, direction changes to L
            ori_r = 0
            ori_c = -1
            row = column + 100
            column = 49
        
        elif column < 0 and 150 <= row < 200 and ori_c == -1: #In face 6, move left to face 1 transpse, direction changes to D
            ori_r = 1
            ori_c = 0
            column = row - 100
            row = 0
           
        elif row >= 200 and 0 <= column < 50 and ori_r == 1: #In face 6, move down to face 2
            row = 0
            column += 100
            
        elif column == 50 and 150 <= row < 200 and ori_c == 1: #In face 6, move right to face 5 transpose, direction changes to U
            ori_r = -1
            ori_c = 0
            column = row - 100
            row = 149
        
            
        if arr[row][column] == "#":
            ori_r = initial_r
            ori_c = initial_c
            row = initial_row
            column = initial_column
            
            return row, column, ori_r, ori_c
        
    return row, column, ori_r, ori_c
        
             

In [ ]:
turn = {
    'R': ('D', 'U'),
    'D': ('L', 'R'),
    'L': ('U', 'D'),
    'U': ('R', 'L')
}
initial_map, instruc, max_l = read_txt(PATH+'/day22.txt')
steps = [int(num) for num in re.findall('\d+', instruc)]
forwards = re.findall('[A-Z]', instruc)

# make each row the same lengh for later transpose
for row in initial_map:
    row = fill_arr(row, max_l - len(row))
    

arr_t = np.array(initial_map).T.tolist()

################################################# PART 1 #################################################

start_r = 0
start_c = initial_map[0].index('.')
start_move = steps.pop(0)

forward = 'R'
cur_r, cur_c = move(start_r, start_c, initial_map, arr_t, forward, start_move, max_l, len(initial_map))

for f, s in zip(forwards, steps):
    forward = turn[forward][0] if f == 'R' else turn[forward][1]
    cur_r, cur_c = move(cur_r, cur_c, initial_map, arr_t, forward, s, max_l, len(initial_map))
    

print(f"row: {cur_r} column: {cur_c} forward: {forward}")
print(f"ans1: {cal_ans1(cur_r, cur_c, forward)}")

################################################# PART 2 #################################################

ori_r = 0
ori_c = 1

cur_r, cur_c = move(start_r, start_c, initial_map, arr_t, 'R', start_move, max_l, len(initial_map))

for f, s in zip(forwards, steps):
    cur_r, cur_c, ori_r, ori_c = move2(cur_r, cur_c, initial_map, f, s, ori_r, ori_c)
    

print(f"row: {cur_r} column: {cur_c} direction: {ori_r, ori_c}")
print(f"ans2: {cal_ans2(cur_r, cur_c, ori_r, ori_c)}")